# 팔머 펭귄

> X를 주면 펭귄을 분류하는 문제

* Iter1. 
    - SVC 같은 선형 분류 모델 하나 선정
    - 그리드 서치
    - 파이프라인 

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("data/data_pipeline/penguins_size.csv")
df.info() # 주니어: 채워널어라 -> 평균값, 앞에 있는 값 | 문자: 채워넣어라 -> 편향
df.sample(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   culmen_length_mm   342 non-null    float64
 3   culmen_depth_mm    342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                334 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
160,Chinstrap,Dream,46.0,18.9,195.0,4150.0,FEMALE
271,Gentoo,Biscoe,48.5,14.1,220.0,5300.0,MALE
86,Adelie,Dream,36.3,19.5,190.0,3800.0,MALE
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,MALE
175,Chinstrap,Dream,50.6,19.4,193.0,3800.0,MALE
111,Adelie,Biscoe,45.6,20.3,191.0,4600.0,MALE
26,Adelie,Biscoe,40.6,18.6,183.0,3550.0,MALE
120,Adelie,Torgersen,36.2,17.2,187.0,3150.0,FEMALE
138,Adelie,Dream,37.0,16.5,185.0,3400.0,FEMALE
48,Adelie,Dream,36.0,17.9,190.0,3450.0,FEMALE


In [4]:
df['species'].unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [5]:
df['sex'].unique() # unique 꼭 찍어봐야 한다: nan, . 전처리 필요

array(['MALE', 'FEMALE', nan, '.'], dtype=object)

In [6]:
df.isnull().sum()

species               0
island                0
culmen_length_mm      2
culmen_depth_mm       2
flipper_length_mm     2
body_mass_g           2
sex                  10
dtype: int64

In [9]:
penguins = df.dropna()

In [11]:
penguins

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [13]:
penguins.reset_index(drop=True, inplace=True)
penguins

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
...,...,...,...,...,...,...,...
329,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE
330,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
331,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
332,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [14]:
penguins['sex'].unique()

array(['MALE', 'FEMALE', '.'], dtype=object)

In [15]:
penguins[penguins.sex == '.']

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
327,Gentoo,Biscoe,44.5,15.7,217.0,4875.0,.


In [16]:
penguins.drop([327], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_5216\1472391410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins.drop([327], inplace=True)


In [17]:
penguins['sex'].unique()

array(['MALE', 'FEMALE'], dtype=object)

In [20]:
penguins['species'] = penguins['species'].astype('category')

C:\Users\user\AppData\Local\Temp\ipykernel_5216\154780174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins['species'] = penguins['species'].astype('category')


In [21]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, 0 to 333
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   species            333 non-null    category
 1   island             333 non-null    object  
 2   culmen_length_mm   333 non-null    float64 
 3   culmen_depth_mm    333 non-null    float64 
 4   flipper_length_mm  333 non-null    float64 
 5   body_mass_g        333 non-null    float64 
 6   sex                333 non-null    object  
dtypes: category(1), float64(4), object(2)
memory usage: 18.7+ KB


In [22]:
penguins['species_cod'] = penguins['species'].cat.codes
penguins

C:\Users\user\AppData\Local\Temp\ipykernel_5216\1339803917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins['species_cod'] = penguins['species'].cat.codes


,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex,species_cod
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE,0
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE,0
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE,0
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE,0
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE,0
...,...,...,...,...,...,...,...,...
329,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE,2
330,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE,2
331,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE,2
332,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE,2


### .astype('category')
### .cat.codes
### 판다스에 있는 기능들이 생각보다 강력하니까 꼭 알아두세요

### SVC(같은 방법)로 해결
### 파이프라인 

In [23]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, 0 to 333
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   species            333 non-null    category
 1   island             333 non-null    object  
 2   culmen_length_mm   333 non-null    float64 
 3   culmen_depth_mm    333 non-null    float64 
 4   flipper_length_mm  333 non-null    float64 
 5   body_mass_g        333 non-null    float64 
 6   sex                333 non-null    object  
 7   species_cod        333 non-null    int8    
dtypes: category(1), float64(4), int8(1), object(2)
memory usage: 19.0+ KB


In [25]:
# 컬럼값 구분 기억 -> 컬럼 순서 기억
penguins_feats = ['island', 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']
penguins_labs = ['species_cod'] # 답안지

In [27]:
X, y = penguins[penguins_feats].values, penguins[penguins_labs].values

##### 1. 데이터 준비
##### 2. 데이터프레임 만들어서 x, y 로 쪼개놓는다
##### 3. 이 데이터를 기반으로 보간을 한다

## SVC 를 사용한 학습 및 평가

In [70]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVC

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

##### 백엔드: 게시판 구현: 1계층 댓글, 무한 댓글: self join
##### split 이후에 정규화한다, 이후 과정을 자동화시키고 싶은 것, 교차검증도 자동화 시키고 싶다.
##### 게시판에 코드 올림: 복습 
##### 백엔드: TODO 포트폴리오 -> 순서 중요

In [71]:
# 정의부
n_feats = [1, 2, 3, 4]
n_tras = Pipeline(steps=[('scaler', StandardScaler())]) # MinMaxScaler, NormalScaler 도 올 수 있다
c_feats = [0, 5]
c_tras = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))]) # onehotencoding 안되면 자잘한 것은 지워라

In [72]:
# 실행부
preprocessor = ColumnTransformer(
    transformers=[
        ('num', n_tras, n_feats), # 1,2,3,4 에 파이프라인 실행해줄래
        ('cat', c_tras, c_feats) # 위에 실행하고 0,5 원핫인코딩
    ]
)

In [73]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor), # divide and conquer
                           ('svc', SVC(probability=True))      
                    ])

##### 데이터분석 -> 전처리   ->   (전처리: Scaler, OneHotEncoding etc) -> 모델 돌리기 -> 교차검증(매개변수 최적화) 
##### [      판다스     ]변환과정 [ 보간 ]   

In [74]:
SVC_scores = []
SVC_fimp = []
SVC_predictions = 0
FOLDS = 5

In [75]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [76]:
for fold, (train_idx, valid_idx) in enumerate(skf.split(penguins[penguins_feats].values, 
                                                        penguins[penguins_labs].values)): # skf: index 가 있는 애들이 들어가야 함
    print(f"Fold = {fold}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 전체 데이터 8:2로 나눈 것
    X_val_train, X_val, y_val_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
    
    multi_model = pipeline.fit(X_val_train, y_val_train) # 교차검증을 파이프라인에 넣으면 안된다: 결과값만 나오고 중간과정을 들여볼 수가 없다.
    preds_valid = pipeline.predict(X_val)
    acc = accuracy_score(y_val, preds_valid)
    SVC_scores.append(acc)

    test_preds = multi_model.predict(X_test)
    SVC_predictions += test_preds/FOLDS

Fold = 0
Fold = 1
Fold = 2
Fold = 3
Fold = 4


c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversio

In [77]:
np.mean(SVC_scores)

0.9850746268656716

In [78]:
SVC_predictions

array([0., 2., 0., 1., 0., 2., 2., 1., 1., 1., 0., 0., 2., 0., 2., 0., 0.,
       1., 0., 2., 0., 0., 2., 1., 0., 0., 2., 2., 1., 2., 1., 2., 0., 0.,
       2., 2., 1., 2., 0., 0., 0., 0., 1., 1., 0., 0., 2., 0., 0., 2., 0.,
       2., 2., 0., 0., 2., 0., 0., 1., 1., 2., 2., 2., 0., 0., 2., 0.])

In [79]:
X_val

array([['Biscoe', 42.6, 13.7, 213.0, 4950.0, 'FEMALE'],
       ['Biscoe', 46.2, 14.5, 209.0, 4800.0, 'FEMALE'],
       ['Dream', 42.5, 16.7, 187.0, 3350.0, 'FEMALE'],
       ['Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE'],
       ['Biscoe', 38.1, 16.5, 198.0, 3825.0, 'FEMALE'],
       ['Dream', 39.6, 18.1, 186.0, 4450.0, 'MALE'],
       ['Dream', 32.1, 15.5, 188.0, 3050.0, 'FEMALE'],
       ['Dream', 52.0, 20.7, 210.0, 4800.0, 'MALE'],
       ['Dream', 46.4, 18.6, 190.0, 3450.0, 'FEMALE'],
       ['Dream', 44.1, 19.7, 196.0, 4400.0, 'MALE'],
       ['Biscoe', 43.3, 13.4, 209.0, 4400.0, 'FEMALE'],
       ['Torgersen', 43.1, 19.2, 197.0, 3500.0, 'MALE'],
       ['Dream', 45.5, 17.0, 196.0, 3500.0, 'FEMALE'],
       ['Biscoe', 37.6, 19.1, 194.0, 3750.0, 'MALE'],
       ['Dream', 39.8, 19.1, 184.0, 4650.0, 'MALE'],
       ['Dream', 55.8, 19.8, 207.0, 4000.0, 'MALE'],
       ['Dream', 38.3, 19.2, 189.0, 3950.0, 'MALE'],
       ['Dream', 46.9, 16.6, 192.0, 2700.0, 'FEMALE'],
       ['Bisc

In [80]:
confusion_matrix(y_val, preds_valid) # row 를 col 로 예측한 값: 1번을 1번으로 분류한 게 13, 이걸 상향시키려면 저 위에 판다스로 가야 한다 -> 전처리하라: 팔머 펭귄 거의 모든 조직에서 전처리 문제를 물어본다

# 최고가 99.98
# 교차검증 파이프라인에 넣으면 안됨, 교차검증 봐두어야 한다
# list comprehension 꼭 공부해야 한다, zip + for, enumerate = for, map = lambda, filter + lambda (파이썬 내장함수)

array([[32,  0,  0],
       [ 1, 13,  0],
       [ 0,  0, 21]], dtype=int64)

In [81]:
preds_valid

array([2, 2, 1, 0, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 0, 1, 0, 1, 0, 2, 1, 2,
       2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1,
       0, 0, 2, 1, 0, 0, 1, 0, 1, 0, 0, 2, 2, 0, 2, 1, 0, 1, 2, 2, 2, 0,
       2], dtype=int8)